In [1]:
! hostname

r002.ib.bridges2.psc.edu


In [15]:
from pathlib import Path
from glob import glob

import duckdb
from pyarrow import dataset as pads
from pyarrow import parquet as pq

In [3]:
con = duckdb.connect()

In [4]:
root = "data"
nrows = 10000
nelem = 100

In [13]:
pq.ParquetDataset

Init signature:
pq.ParquetDataset(
    path_or_paths=None,
    filesystem=None,
    schema=None,
    metadata=None,
    split_row_groups=False,
    validate_schema=True,
    filters=None,
    metadata_nthreads=None,
    read_dictionary=None,
    memory_map=False,
    buffer_size=0,
    partitioning='hive',
    use_legacy_dataset=None,
    pre_buffer=True,
    coerce_int96_timestamp_unit=None,
    thrift_string_size_limit=None,
    thrift_container_size_limit=None,
)
Docstring:     
Encapsulates details of reading a complete Parquet dataset possibly
consisting of multiple files and partitions in subdirectories.

Parameters
----------
path_or_paths : str or List[str]
    A directory name, single file name, or list of file names.
filesystem : FileSystem, default None
    If nothing passed, will be inferred based on path.
    Path will try to be found in the local on-disk filesystem otherwise
    it will be parsed as an URI to determine the filesystem.
metadata : pyarrow.parquet.FileMetaDa

In [16]:
datadir = Path(root) / f"nrows-{nrows}_nelem-{nelem}"
attrib = pads.dataset(str(datadir / "attrib.parquet"))
arrays = pads.dataset(list(sorted(glob(str(datadir / "arrays-*.parquet")))))

In [19]:
df = con.query("SELECT * from attrib where attrib.attrib < 0.1").df()

In [20]:
df.head()

,attrib,__index_level_0__
0,0.029850,16
1,0.064889,21
2,0.037918,22
3,0.050138,28
4,0.063490,41


In [22]:
sql = """SELECT 
    attrib.__index_level_0__, arrays.__index_level_0__, attrib.attrib, arrays.array
from attrib
left join arrays
    on attrib.__index_level_0__ = arrays.__index_level_0__
where attrib.attrib < 0.1
"""

In [27]:
threshold = 0.1

sql = f"""
with filtered as (
    select * from attrib where attrib.attrib < {threshold:.6f}
)
select
    filtered.__index_level_0__, arrays.__index_level_0__, filtered.attrib, arrays.array
from filtered 
left join arrays
    on filtered.__index_level_0__ = arrays.__index_level_0__
"""

In [28]:
df = con.query(sql).df()

In [29]:
df.head()

,__index_level_0__,__index_level_0___2,attrib,array
0,16,16,0.029850,"[-0.3740611486267887, 0.701022828054255, -1.91..."
1,21,21,0.064889,"[-0.12444886848162703, 0.7523795492775105, 1.7..."
2,28,28,0.050138,"[-2.1507090466901246, -0.5456877844872032, -0...."
3,41,41,0.063490,"[-0.05590238127902171, 0.22616237487552604, -0..."
4,42,42,0.064209,"[0.5239562476585321, 1.114994197727871, 0.2957..."
